<a href="https://colab.research.google.com/github/Lukaid/my_notebook/blob/main/blog_scraping_using_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_posts(url):

    search_url = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(search_url, 'html.parser')
    blog_url = 'https://blog.naver.com' + soup.select('iframe#mainFrame')[0].get('src')
    post = urllib.request.urlopen(blog_url).read()
    post_html = BeautifulSoup(post, 'html.parser')
    post_content_text = ""

    for post_content in post_html.find_all('div', {'class':'se-main-container'}):
        post_content_text += post_content.get_text()

    return post_content_text

In [5]:
import os
import sys
import urllib.request
import pandas as pd
import json
import re

client_id = "VzhDPPl2jYhHHqnnCnXG"
client_secret = "XD20x9HONl"

Qust_list = ['쿠팡 배송', '쿠팡 리뷰', '롯데on', 'ssg.com', '쓱닷컴']

for Q_query in Qust_list:
    query = urllib.parse.quote(Q_query)
    # encText = urllib.parse.quote("컴퓨터") # quote가 문자열을 utf-8으로 변경해줌
    idx = 0
    display = 100 # 가져올 정보의 개수 , default = 10, max = 100
    start = 1
    end = 1000
    sort = "sim"

    blog_df = pd.DataFrame(columns = ('Title', 'Link', 'Description', 'Blogger Name', 'Blogger Link', 'Post Data', 'Post Content'))

    for start_index in range(start, end, display):

        url = "https://openapi.naver.com/v1/search/blog?query=" + query \
        + "&display=" + str(display) \
        + "&start=" + str(start_index) \
        + "&sort=" + sort

        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if (rescode==200): # 정상코드
            response_body = response.read()
            response_dict = json.loads(response_body.decode('utf-8'))
            items = response_dict['items']
            for item_index in range(0, len(items)):
                remove_tag = re.compile('<.*?>')
                title = re.sub(remove_tag, '', items[item_index]['title'])
                link = items[item_index]['link']
                description = re.sub(remove_tag, '', items[item_index]['description'])
                blogger_name = items[item_index]['bloggername']
                blogger_link = items[item_index]['bloggerlink']
                post_data = items[item_index]['postdate']
                try:
                    post_content = get_posts(link)
                except:
                    pass

                blog_df.loc[idx] = [title, link, description, blogger_name, blogger_link, post_data, post_content]
                idx += 1
        else:
            print("Error Code:" + rescode)

    blog_df.to_csv(f'blog_df_{Q_query}.csv', encoding='utf-8-sig')



검색 질의: 쿠팡 리뷰


,Title,Link,Description,Blogger Name,Blogger Link,Post Data,Post Content
0,"애드포스트, 쿠팡파트너스, 리뷰업체 기자단, 당근마켓...",https://blog.naver.com/minsoism?Redirect=Log&l...,그런 면에서 쿠팡 미국 나스닥 상장을 축하하며 앞으로 더 잘됐으면 좋겠다. 어쨌거나...,유후엄마 가장 보통의 육아,https://blog.naver.com/minsoism,20210215,\n\n\n\n\n올해는 매달 초에 지난달의 결산을 올리며 마음을 다잡으려고 했는데...
1,가성비 핫팩 대량 구매 [내돈내산 그냥저냥 리뷰_ 쿠팡편],https://blog.naver.com/leemoc1008?Redirect=Log...,- 쿠팡 or 네이버에서 직접구매한 제품 리뷰 -길게 안씁니다.... 이번 2020...,SIK.Community,https://blog.naver.com/leemoc1008,20210125,\n\n\n\n\n- 쿠팡 or 네이버에서 직접구매한 제품 리뷰 -길게 안씁니다....
2,[프레시지] 신림동 백순대 볶음 쿠팡밀키트 솔직리뷰,https://blog.naver.com/dew135?Redirect=Log&log...,전자렌지 1분 아님! 이상 프레시지 신림동 백순대볶음 리뷰 끝. #쿠팡밀키트 #프레...,베리위크,https://blog.naver.com/dew135,20210302,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​fresheasy[프레시...
3,쿠팡 탐사 미세먼지 마스크 리뷰,https://blog.naver.com/toto2700?Redirect=Log&l...,#쿠팡 #리뷰 #리뷰어 #미세먼지마스크 #KF80 #KF94 안녕하세요 여러분 귤...,귤덕이네,https://blog.naver.com/toto2700,20190329,\n\n\n\n#쿠팡 #리뷰 #리뷰어 #미세먼지마스크 #KF80 #KF94​안녕하세...
4,"책 리뷰 | 쿠팡 셀러되기 , 쿠팡 판매자를 위한 기본기 및...",https://blog.naver.com/dkf4321?Redirect=Log&lo...,책 리뷰 | 쿠팡 셀러되기 지금은 쿠팡시대!? 쿠팡 셀러가 되기 위한 기본지식부터 ...,리앙이의 소소한 공간,https://blog.naver.com/dkf4321,20210325,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\...
...,...,...,...,...,...,...,...
95,쿠팡쇼핑리뷰 - 콘삭 커피,https://blog.naver.com/love5637?Redirect=Log&l...,페이퍼필터원두 커피 콘삭 오늘의 리뷰 #베트남커피 #콘삭커피 일명 다람쥐커피 라고 ...,소소한일상이야기♥,https://blog.naver.com/love5637,20190131,\n\n\n\n\n\n\n\n\n\n콘삭 페이퍼 필터 퓨어 아라비카 100% 커피\...
96,세미 뽑아쓰는 수세미 리뷰 : 쿠팡 로켓배송,https://blog.naver.com/eunko16?Redirect=Log&lo...,그렇다면 개인리뷰 들어갈게요! * 지극히 개인적인 수세미 장점 * - 세균 걱정없이...,"우린, 사랑아니면 여행이겠지",https://blog.naver.com/eunko16,20180503,
97,[쿠팡] 스피카 여성용 요가복 구입 후에 리뷰 남겨요,https://chanyong.tistory.com/877,(후기 건수) 7개 상품평(🖱전체보기) (🏆 프리미엄 리뷰) (평가) ⭐⭐⭐⭐⭐ 좋...,나의 쇼핑 라이프,https://chanyong.tistory.com/,20210324,
98,털실리뷰 : 쿠팡직구로산 싸고 양많은 털실 퍼즐얀,https://blog.naver.com/powerppodea?Redirect=Lo...,쓰셔도 될것 같아요 ^^ 한타래로 네키목도리 하나정도는 나올것 같습니다! 아참! 그...,김라희 블로그,https://blog.naver.com/powerppodea,20181001,
